In [1]:
import toml
import json
import millify
import gspread
import requests
import calendar
import numpy as np
import pandas as pd
import polars as pl
import datetime as dt
import streamlit as st
from millify import prettify
import chart_functions as chart
from lxml.html import fromstring
from streamlit_gsheets import GSheetsConnection
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials

import schemas
import chart_functions

In [2]:
def load_environment_variables(secret_path):
    return toml.load(secret_path)

def authenticate(secrets, scope, workbook_name):
    credentials_file = json.loads(str(environment['connections']['gsheets']).replace("'", '"').replace('\r\n', '\\r\\n'))
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(credentials_file, scopes=scope)
    client = gspread.authorize(credentials)
    wb = client.open(workbook_name)
    return wb

def pad_data(data, length):
    padded_data = [[None if x == "" else x for x in row] + [None] * (length - len(row)) for row in data]
    # for i in range(len(data)):
        # data[i] = [None if x == "" else x for x in data[i]]
        # data[i] = [row + [None] * (length - len(row)) for row in data]
    return padded_data

def load_data(sheet_name, schema):
    sheet = WORKBOOK.worksheet(sheet_name)
    data = sheet.get()
    headers = data[0]
    padded_data = pad_data(data[1:], len(headers))
    loaded_dataframe = pl.DataFrame(padded_data, schema=schema, orient='row', strict=False)
    return loaded_dataframe

environment = load_environment_variables('.streamlit/secrets.toml')
scope = environment['scopes']['scope']
WORKBOOK = authenticate(environment, scope, 'NLFB')

In [24]:
month_num_from_name_dict = {name:num for num, name in enumerate(calendar.month_name) if num}

In [27]:
environment = load_environment_variables('.streamlit/secrets.toml')
type(environment['connections']['gsheets'])

dict

In [13]:
resources_data.filter(pl.col('Resource') == 'Meetup Page')['URL'][0]

'https://www.meetup.com/20-and-30-somethings-book-club-london/'

In [ ]:
df.head()

Number,ISBN,Month,Year,Title,Score,Author,Publisher,Pages,Author gender,Pub year,Goodreads score,Our score conversion,variance,Debut?,Translated?,Topics
i64,str,str,i64,str,f64,str,str,i64,str,i64,f64,f64,f64,str,str,str
34,"""9780571376483""","""June""",2024,"""Demon Copperhead""",8.192,"""Barbara Kingsolver""","""Faber & Faber""",560,"""Female""",2023,null,4.096,4.096,"""no""","""no""",null
14,"""9781784744649""","""November""",2022,"""Tomorrow, and Tomorrow, and To…",7.84,"""Gabrielle Zevin""","""Vintage Publishing""",416,"""Female""",2022,4.22,3.92,-0.3,"""no""","""no""",null
31,"""9781838930509""","""March""",2024,"""Pachinko""",7.7,"""Min Jin Lee""","""Bloomsbury Publishing PLC""",560,"""Female""",2017,null,3.85,3.85,"""no""","""no""","""Romance, Crime, War, Music, Pe…"
22,"""9780008532772""","""July""",2023,"""Yellowface""",7.66,"""Rebecca F. Kuang""","""HarperCollins Publishers""",336,"""Female""",2023,4.04,3.83,-0.21,"""no""","""no""",null
30,"""9781529111798""","""February""",2024,"""I who have never known men""",7.51,"""Jacqueline Harpman""","""Vintage Publishing""",208,"""Female""",1997,null,3.755,3.755,"""no""","""yes""",null


In [3]:
import utils

resources_data = utils.load_data('Resources', schema=schemas.resources_schema, workbook=WORKBOOK)
meetup_url = resources_data.filter(pl.col('Resource') == 'Meetup Page')['URL'][0]
members = utils.get_number_of_members(meetup_url, "member-count-link")



In [4]:
print(members)

5,970 members · Public group


In [7]:
int(members.split(' ')[0].replace(',', ''))

5970

In [8]:
'5783'.replace(',', '')

'5783'